## Install prerequisites

In [1]:
import time
import humanize
start_time = time.time()

In [2]:
# install Open Graph Benchmark
! pip install ogb

# install PyTorch Geometric
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

     |████████████████████████████████| 45 kB 1.6 MB/s eta 0:00:011
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 12.3 MB 20.6 MB/s eta 0:00:01
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 21.6 MB 7.7 MB/s eta 0:00:01    |████████████▎                   | 8.3 MB 7.7 MB/s eta 0:00:02     |██████████████▍                 | 9.7 MB 7.7 MB/s eta 0:00:02
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 18.2 MB 13.3 MB/s eta 0:00:01
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 6.3 MB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 153 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 231 kB 23.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 22.7 MB/s eta 0:00:01     |███                             | 

  Created wheel for torch-geometric: filename=torch_geometric-1.5.0-py3-none-any.whl size=267915 sha256=90a10e4560e0bb58e9582efe3e2f09e6399d4aa7d4c1641c5c22890b193bea37
  Stored in directory: /root/.cache/pip/wheels/87/09/84/1d026c4e02fc66af6224faaa7c35db83a178ec6593e69baf06
Successfully built torch-geometric


## Import libraries

In [3]:
import torch
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from torch_sparse import SparseTensor
from torch_scatter import scatter
from torch_geometric.nn.inits import glorot, zeros

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

## Logger

In [4]:
class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

## Config

In [17]:
class args:
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    log_steps = 1
    num_layers = 3
    skip_layers = 3
    hidden_channels = 256
    dropout = 0.05
    lr = 0.005
    epochs = 1000
    eval_steps = 10
    runs = 1

In [6]:
device = torch.device(args.device)

dataset = PygNodePropPredDataset(name='ogbn-proteins')
split_idx = dataset.get_idx_split()
data = dataset[0]

x = scatter(data.edge_attr, data.edge_index[0], dim=0,
            dim_size=data.num_nodes, reduce='mean').to(device)

y_true = data.y.to(device)
train_idx = split_idx['train'].to(device)

edge_index = data.edge_index.to(device)
adj = SparseTensor(row=edge_index[0], col=edge_index[1])

Downloaded 0.21 GB: 100%|██████████| 216/216 [00:53<00:00,  4.03it/s]


Extracting dataset/proteinfunc.zip
Processing...
Loading necessary files...
This might take a while.


  0%|          | 0/1 [00:00<?, ?it/s]

Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 240.18it/s]


Converting graphs into PyG objects...
Saving...
Done!


In [7]:
print(humanize.naturaldelta(start_time - time.time()))

4 minutes


## Model Definition

In [8]:
from torch_geometric.nn import DenseSAGEConv
normalize = True


In [9]:
class GCNConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.weight = Parameter(torch.Tensor(in_channels, out_channels))
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        

    def forward(self, x, adj):
        return adj @ x @ self.weight

In [10]:
class SAGEConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.weight = Parameter(torch.Tensor(in_channels, out_channels))
        self.root_weight = Parameter(torch.Tensor(in_channels, out_channels))
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        glorot(self.root_weight)
        zeros(self.bias)

    def forward(self, x, adj):
        out = adj.matmul(x, reduce="mean") @ self.weight
        out = out + x @ self.root_weight + self.bias
        return out

In [11]:
class ResidualGC(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, skip_n_channels,dropout):
        super(ResidualGC, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))
        for _ in range(skip_n_channels):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        self.compress = SAGEConv(hidden_channels + in_channels, hidden_channels)
        
        self.convs.append(GCNConv(hidden_channels, out_channels))
        
        self.dropout = dropout
    
    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        self.compress.reset_parameters()
            
    def forward(self, x, adj):
        x_i = x #identity
        for conv in self.convs[:-1]:
            x = conv(x, adj)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = torch.cat([x_i,x],1)
        
        x = self.compress(x, adj)
        x = self.convs[-1](x, adj)
        return x
    

In [12]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()
        self.init_conv = GCNConv(in_channels, hidden_channels)
        self.res1 = ResidualGC(hidden_channels,hidden_channels, hidden_channels,3, dropout)    
        self.bridge = GCNConv(hidden_channels, hidden_channels)     
        self.res2 = ResidualGC(hidden_channels,hidden_channels, out_channels,3, dropout)
        self.final_conv = GCNConv(out_channels, out_channels)    
        self.dropout = dropout

    def reset_parameters(self):
        self.init_conv.reset_parameters()
        self.res1.reset_parameters()
        self.bridge.reset_parameters()
        self.res2.reset_parameters()
        self.final_conv.reset_parameters()

    def forward(self, x, adj):
        x = self.init_conv(x,adj)
        x = self.res1(x,adj)
        x = self.bridge(x,adj)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.res2(x,adj)
        x = self.final_conv(x,adj)

        return x

In [ ]:
class RRGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(RRGCN, self).__init__()
        self.init_conv = GCNConv(in_channels, hidden_channels)
        self.res1 = ResidualGC(hidden_channels,hidden_channels, hidden_channels,3, dropout)    
        self.bridge = GCNConv(hidden_channels, hidden_channels)     
        self.res2 = ResidualGC(hidden_channels,hidden_channels, hidden_channels,3, dropout)
        self.compress = GCNConv(hidden_channels*2, out_channels)
        self.final_conv = GCNConv(out_channels, out_channels)    
        self.dropout = dropout

    def reset_parameters(self):
        self.init_conv.reset_parameters()
        self.res1.reset_parameters()
        self.bridge.reset_parameters()
        self.res2.reset_parameters()
        self.final_conv.reset_parameters()

    def forward(self, x, adj):
        x = self.init_conv(x,adj)
        x_i = x
        x = self.res1(x,adj)
        x = self.bridge(x,adj)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.res2(x,adj)
        x = torch.cat([x,x_i],1) 
        x = self.final_conv(x,adj)

        return x

## Training and eval

In [13]:
def train(model, x, adj, y_true, train_idx, optimizer):
    model.train()
    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer.zero_grad()
    out = model(x, adj)[train_idx]
    loss = criterion(out, y_true[train_idx].to(torch.float))
    loss.backward()
    optimizer.step()

    return loss.item()

In [14]:
@torch.no_grad()
def test(model, x, adj, y_true, split_idx, evaluator):
    model.eval()

    y_pred = model(x, adj)

    train_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['rocauc']
    valid_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['rocauc']
    test_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc

In [15]:
model = GCN(x.size(-1), args.hidden_channels, 112, args.num_layers,args.dropout).to(device)

# model = ResidualGC(x.size(-1),args.hidden_channels,112,3,args.dropout).to(device)


In [19]:

# Pre-compute GCN normalization.
adj = adj.set_diag()
deg = adj.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj = deg_inv_sqrt.view(-1, 1) * adj * deg_inv_sqrt.view(1, -1)
    
evaluator = Evaluator(name='ogbn-proteins')
logger = Logger(args.runs, args)
best_test_score = 0

for run in range(args.runs):
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    for epoch in range(1, 1 + args.epochs):
        
        loss = train(model, x, adj, y_true, train_idx, optimizer)

        if epoch % args.eval_steps == 0:
            result = test(model, x, adj, y_true, split_idx, evaluator)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:                
                train_rocauc, valid_rocauc, test_rocauc = result
                print(f'Run: {run + 1:02d}\t '
                      f'Epoch: {epoch:02d}\t '
                      f'Loss: {loss:.4f}\t '
                      f'Train: {100 * train_rocauc:.2f}%\t '
                      f'Valid: {100 * valid_rocauc:.2f}%\t'
                      f'Test: {100 * test_rocauc:.2f}%')
                if(test_rocauc > best_test_score):
                    best_test_score = test_rocauc
                    save_path = "gcn_" + f'Test: {100 * test_rocauc:.2f}' + ".pth"
                    torch.save(model, save_path)
                    print("Model saved.")

    logger.print_statistics(run)
logger.print_statistics()

Run: 01	 Epoch: 10	 Loss: 0.5052	 Train: 54.04%	 Valid: 47.02%	Test: 42.46%
Model saved.
Run: 01	 Epoch: 20	 Loss: 0.3479	 Train: 66.58%	 Valid: 58.08%	Test: 50.57%
Model saved.
Run: 01	 Epoch: 30	 Loss: 0.3297	 Train: 69.02%	 Valid: 65.42%	Test: 57.40%
Model saved.
Run: 01	 Epoch: 40	 Loss: 0.3230	 Train: 70.39%	 Valid: 67.40%	Test: 61.99%
Model saved.
Run: 01	 Epoch: 50	 Loss: 0.3198	 Train: 71.48%	 Valid: 67.19%	Test: 60.97%
Run: 01	 Epoch: 60	 Loss: 0.3170	 Train: 72.22%	 Valid: 68.85%	Test: 63.13%
Model saved.
Run: 01	 Epoch: 70	 Loss: 0.3118	 Train: 73.42%	 Valid: 70.66%	Test: 61.53%
Run: 01	 Epoch: 80	 Loss: 0.3070	 Train: 74.70%	 Valid: 71.81%	Test: 64.32%
Model saved.
Run: 01	 Epoch: 90	 Loss: 0.3035	 Train: 75.97%	 Valid: 73.51%	Test: 65.35%
Model saved.
Run: 01	 Epoch: 100	 Loss: 0.3092	 Train: 75.05%	 Valid: 72.72%	Test: 67.62%
Model saved.
Run: 01	 Epoch: 110	 Loss: 0.3015	 Train: 76.45%	 Valid: 73.23%	Test: 66.97%
Run: 01	 Epoch: 120	 Loss: 0.2952	 Train: 77.83%	 Valid: 7

In [20]:
print(humanize.naturaldelta(start_time - time.time()))

an hour
